# Grid search as initial guess for optimisation

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from acrobotics.planning import cart_to_joint_no_redundancy
from acrobotics.planning import get_shortest_path
from acrobotics.util import get_default_axes3d
from acrobotics.recources.robots import Kuka
from acrobotics.recources.path_on_table import path_ori_free, scene1, scene2, path_ori_free

robot = Kuka()
scene = scene2
ns = 200 # number of samples for every path point

## Calculate intial solution using grid search

In [3]:
time_before = time.time()

Q = cart_to_joint_no_redundancy(robot, path_ori_free, scene, num_samples=ns)
print([len(qi) for qi in Q])
res = get_shortest_path(Q, method='dijkstra')

run_time_grid = time.time() - time_before

Processing point 1/15
Processing point 2/15
Processing point 3/15
Processing point 4/15
Processing point 5/15
Processing point 6/15
Processing point 7/15
Processing point 8/15
Processing point 9/15
Processing point 10/15
Processing point 11/15
Processing point 12/15
Processing point 13/15
Processing point 14/15
Processing point 15/15
[742, 348, 116, 36, 58, 86, 222, 400, 404, 380, 148, 64, 36, 16, 16]


In [4]:
qp_init = np.array(res['path'])

Show the setup. The magic command `%matplotlib qt` makes that the plot is created in a separate (Qt) window.

In [5]:
# %matplotlib qt
# fig2, ax2 = get_default_axes3d([-1, 1], [-1, 1], [-1, 1])
# for pi in path_ori_free: pi.plot(ax2)
# scene.plot(ax2, c='g')
# robot.animate_path(fig2, ax2, qp_init)
# plt.show(block=True)

## Formulate the optimisation problem

First let's define some constants based on the robot and scene we want to plan for:

- **N**: Path descritization, number of path points.
- **S**: Number of planes used to describe the collision shapes, in this case only boxes.
- **ndof**: Degrees of freedom of the robot. We assume the robot has only one collision shape for each link.
- **nobs**: The number of obstacles in the scene.
- **eps**: Minimum distance to consider two obstacles not in collision.

In [6]:
N, S, ndof, nobs, eps = len(path_ori_free), 6, robot.ndof, len(scene.shapes), 1e-6

Fromulate the same task as explicit numpy arrays for the desired end-effector positions along the path:

In [7]:
xp = np.ones(N) * 0.8
yp = np.linspace(-0.2, 0.2, N)
zp = np.ones(N) * 0.2

Calculate the matrix $\mathbf{A}$ and vector $\mathbf{b}$ representing the polyhedron for every collision shape in the problem.

In [8]:
pol_mat_a_robot = []
pol_mat_b_robot = []

# assume all links have only one shape
# expressed in the local link frame
for link in robot.links:
    s = link.geometry.shapes[0]
    Ai, bi = s.get_polyhedron(np.eye(4))
    pol_mat_a_robot.append(Ai)
    pol_mat_b_robot.append(bi)

pol_mat_a_scene , pol_mat_b_scene = scene.get_polyhedron()

Optimisation variables:

- **q**: (N, ndof) matrix representing the descritized robot path.
- **$\lambda$ and $\mu$**: dual variables for the collision constraints. These are a 3D list of vectors of dimension (S, 1). Every combination of robot shape and obstacle shape has such a vector, for every point along the path where the collision constraints are enforced.

In [9]:
import casadi as ca

opti = ca.Opti()

q = opti.variable(N, 6)  #  joint variables along path

# dual variables arranged in convenient lists to acces with indices
lam = [[[opti.variable(S) for j in range(nobs)] for i in range(ndof)] for k in range(N)]
mu =  [[[opti.variable(S) for j in range(nobs)] for i in range(ndof)] for k in range(N)]

Now probably the most complicated part, formulate the collision constrainst for every combination of robot and scene shape, for every point along the path.

In [10]:
from casadi import dot
from acrobotics.optimization import fk_all_links

def col_con(lam, mu, Ar, Ao, br, bo):
    opti.subject_to( -dot(br, lam) - dot(bo, mu) >= eps)
    opti.subject_to(        Ar.T @ lam + Ao.T @ mu == 0.0)
    opti.subject_to( dot(Ar.T @ lam, Ar.T @ lam) <= 1.0)
    opti.subject_to(                         lam >= 0.0)
    opti.subject_to(                          mu >= 0.0)


for k in range(N):
    fk = fk_all_links(robot.links, q[k, :])
    for i in range(ndof):
        Ri = fk[i][:3, :3]
        pi = fk[i][:3, 3]
        for j in range(nobs):
            Ar = pol_mat_a_robot[i] @ Ri.T
            br = pol_mat_b_robot[i] + Ar @ pi
            col_con(lam[k][i][j], mu[k][i][j],
                    Ar,
                    pol_mat_a_scene[j],
                    br,
                    pol_mat_b_scene[j])

Finally the objective and path constraints.

In [11]:
from acrobotics.optimization import fk_kuka2

V = ca.sum1( ca.sum2(  (q[:-1, :] - q[1:, :])**2  ) )# + 0.05* ca.sumsqr(q) #+ 1 / ca.sum1(q[:, 4]**2)
opti.minimize(V)

for i in range(N):
    Ti = fk_kuka2(q[i, :])
    opti.subject_to(xp[i] == Ti[0, 3])
    opti.subject_to(yp[i] == Ti[1, 3])
    opti.subject_to(zp[i] == Ti[2, 3])

## Solve the optimisation problem

I added the magic command `%%capture` to suppress all the solver output.

In [15]:
%%capture

opti.solver('ipopt')
# opti.set_initial(q, qp_init)
opti.set_initial(q, np.zeros((N, ndof)))

time_before = time.time()
sol = opti.solve()
run_time = time.time() - time_before

Get the solution and calculate the cost

In [16]:
qp_sol = opti.value(q)
cost = np.sum((qp_sol[:-1, :] - qp_sol[1:, :])**2)
print("Cost: {}".format(cost))
print("Runtime: {}".format(run_time))

Cost: 1.3280042372667498
Runtime: 122.24379014968872


Animate the solution!

In [17]:
%matplotlib qt
fig, ax = get_default_axes3d([0, 1], [-0.5, 0.5], [0, 1])
ax.scatter(xp, yp, zp)
scene.plot(ax, c='g')
robot.animate_path(fig, ax, qp_sol)
ax.set_axis_off()
ax.view_init(24, 50)
plt.show()

## Save results

In [15]:
# data (copy-pasted) from run with 200 samples / path point
data = {'grid_sol': res, 'grid_time': run_time_grid,
        'opti_time': run_time, 'opti_cost': cost, 'opti_sol': qp_sol}

In [16]:
# np.save('case_2_data_100_combined', data)
np.save('case_{}_data_{}_combined'.format(2, ns), data)
# np.save('case_2_data_500_combined', data)